In [1]:
import numpy as np
import pandas as pd

In [2]:
def flattend_predictions(pred_input):
        out_flatten = []
        for list_elt in pred_input:
            for elt in list_elt:
                out_flatten.append(elt)
        return out_flatten

def map_predictions_TTA(predictions, labels_map, thresholds):
    """
    Return the predictions mapped to their labels
    :param predictions: the predictions from the predict() method
    :param labels_map: the map
    :param thresholds: The threshold of each class to be considered as existing or not existing
    :return: the predictions list mapped to their labels
    """
    predictions_labels = []
    for image_pred in predictions:
        list_labels = map_predictions(image_pred, labels_map, thresholds)
        list_flatten = flattend_predictions(list_labels)
        labels = []
        for nb, label in labels_map.iteritems():
            if list_flatten.count(label)/float(len(labels_map)) > 0.5:
                labels.append(label)
        predictions_labels.append(labels)    
    return predictions_labels
    
def map_predictions(predictions, labels_map, thresholds):
    """
    Return the predictions mapped to their labels
    :param predictions: the predictions from the predict() method
    :param labels_map: the map
    :param thresholds: The threshold of each class to be considered as existing or not existing
    :return: the predictions list mapped to their labels
    """
    predictions_labels = []
    for prediction in predictions:
        labels = [labels_map[i] for i, value in enumerate(prediction) if value > thresholds[i]]
        predictions_labels.append(labels)

    return predictions_labels

In [14]:
threshold_0 = np.load("/home/jb/amazon/notebooks/threshold0.npy")
threshold_1 = np.load("/home/jb/amazon/notebooks/threshold1.npy")
threshold_2 = np.load("/home/jb/amazon/notebooks/threshold2.npy")
threshold_3 = np.load("/home/jb/amazon/notebooks/threshold1.npy")
#threshold_4 = np.load("/home/jb/amazon/notebooks/threshold2.npy")
thresholds = [0] * 17
for i in range(17):
    thresholds[i] = (threshold_0[i] + threshold_1[i] + threshold_2[i] + threshold_3[i]) / 4.0
    #threshold[i] = (threshold_0[i] + threshold_1[i] + threshold_2[i] + threshold_3[i] + threshold_4[i]) / 5.0

In [15]:
thresholds

[0.072500000000000009,
 0.0275,
 0.6875,
 0.12,
 0.0625,
 0.22249999999999998,
 0.50250000000000006,
 0.54499999999999993,
 0.034999999999999996,
 0.054999999999999993,
 0.17999999999999999,
 0.0625,
 0.14999999999999999,
 0.060000000000000005,
 0.16750000000000001,
 0.042500000000000003,
 0.35749999999999998]

In [9]:
y_map = {0: 'agriculture',
 1: 'artisinal_mine',
 2: 'bare_ground',
 3: 'blooming',
 4: 'blow_down',
 5: 'clear',
 6: 'cloudy',
 7: 'conventional_mine',
 8: 'cultivation',
 9: 'habitation',
 10: 'haze',
 11: 'partly_cloudy',
 12: 'primary',
 13: 'road',
 14: 'selective_logging',
 15: 'slash_burn',
 16: 'water'}

In [10]:
resnet_no_val = np.load("/home/jb/amazon/proba_file_ResNet50_no_val.npy")
vgg19_no_val = np.load("/home/jb/amazon/proba_file_VGG19_no_val.npy")

In [11]:
resnet_vgg19 = np.concatenate((resnet_no_val, vgg19_no_val), axis=1)

In [16]:
predicted_labels = map_predictions_TTA(resnet_vgg19, y_map, thresholds)

In [12]:
x_test_filename_tot = np.load('/home/jb/amazon/x_test_filename_tot.npy')

In [17]:
tags_list = [None] * len(predicted_labels)
for i, tags in enumerate(predicted_labels):
    tags_list[i] = ' '.join(map(str, tags))

#x_test_filename_tot = np.hstack((x_test_filename, x_test_filename_additional))
final_data = [[filename.split(".")[0], tags] for filename, tags in zip(x_test_filename_tot, tags_list)]

In [18]:
final_df = pd.DataFrame(final_data, columns=['image_name', 'tags'])
final_df.to_csv("/home/jb/amazon/ensemble_threshold.csv", index=False)